In [5]:

# TODO change the get_data function by creating an empty list and keep filling in to avoid checking in the end for an unanswered questions
# TODO figure out how to deal with exceptions 
# TODO clean the data to be more presentable  
# TODO identify the source of slow performance
# TODO figure out how to remove special characters
# TODO analyze the data and perform visualization

In [6]:
%reset -f
# The above magic command reset the variable, -f denote no user confirmation 

In [7]:
import requests
import praw
from praw.models import MoreComments 
import pandas as pd
from collections import defaultdict
from bs4 import BeautifulSoup
import json
import re

In [8]:
%load_ext blackcellmagic

The blackcellmagic extension is already loaded. To reload it, use:
  %reload_ext blackcellmagic


In [9]:
def main():
    """
    docstring
    """
    json_file = "reddit_api_config.json"
    json_key = "reddit_api"
    urls = [
        "https://www.reddit.com/r/Accounting/comments/o6c81e/pwc_2021_compensation_thread/",
        # "https://www.reddit.com/r/Accounting/comments/hb5u1h/pwc_2020_compensation_thread/",
        # "https://www.reddit.com/r/Accounting/comments/c1ob0e/pwc_2019_compensation_thread/", 
    ]
    for url in urls:
        reddit = authenticate(json_file=json_file, json_key=json_key)
        submission = reddit.submission(url=url)
        print(f"working on {url}")
        data_dict = get_data(submission)
        file_name = get_filename(submission)
        save_data(data_dict, file_name)
    return data_dict

In [10]:
def authenticate(json_file, json_key):
    print("Authentication on going.....")
    # open the json file and load the data as dictionary
    with open(json_file) as f:
        data = json.load(f)
    # look into the schema of json_key in json_file
    user_values = data[json_key]

    # initiate the session
    reddit = praw.Reddit(
        client_id=user_values["client_id"],
        client_secret=user_values["client_secret"],
        user_agent=user_values["user_agent"],
    )
    
    print("Success! Authenticated")
    return reddit

In [11]:
def check_authentication(reddit):
    # Test if the read only instance was created properly 
    print(reddit.read_only)

In [12]:
def get_filename(submission):
    """
    docstring
    """
    file_name = submission.title.replace(" ", "_")
    file_name = f"{file_name}.csv"
    return file_name


In [13]:
def get_category(submission):
    # obtaining the body of the OP's text in html format
    submission_text = submission.selftext_html
    # create soup object 
    soup = BeautifulSoup(submission_text, 'html.parser')
    # the list is always in ordered list, or ol. hence find that 
    category_list = soup.find("ol")
    # we category_list.text will provide the str of category, hence split to make a list 
    category_list = category_list.text.split("\n")
    # remove the empty elements in the list 
    category_list = list(filter(None,category_list))
    # numbering for the category list
    return category_list


In [14]:
def get_number (category_list):
    """
    docstring
    """
    return range(1, len(category_list)+1)
# print(lookup_dict)



In [15]:
def get_lookup_dict(num_list,category_list):
    """
    docstring
    """
    # create lookup dictionary to identify category based on the number 
    return dict(zip(num_list, category_list))


In [16]:
def get_check_dict(num_list):
    """
    docstring
    """
    # this dictionary will keep track of which items are responded
    return {i:False for i in num_list}
    

In [17]:
def get_data(submission):
    print("populating the lists")
    category_list = get_category(submission=submission)
    num_list = get_number(category_list=category_list)
    check_dict = get_check_dict(num_list=num_list)
    lookup_dict = get_lookup_dict(num_list=num_list, category_list=category_list)
    submission_dict = defaultdict(list)
    print("Lists populated")
    
    # for each comment in the comments object
    print("Starting the loop")
    for comment in submission.comments: 
        # if the current comment is MoreComments objects, skip 
        if isinstance(comment, MoreComments):
            continue
        # obtain the comment body 
        content = comment.body
        # split the contents line by line  
        content = content.split("\n")
        # remove empty strings from the list 
        content = list(filter(None, content))
        # only select line that start with the numbers 
        content = [i for i in content if i[0].isnumeric()]
        # if the list is blank, skip
        if not content: 
            continue
        # create list of numbers from dict keys
        key_list = str(list(lookup_dict.keys()))
        # for each line in the content, or the comment 
        for line in content:
            # the number is always the first character 
            heading_number = float(line[0])
            # obtain the category using the look up table 
            try:
                if check_dict[heading_number]: 
                    continue
            except KeyError:
                continue
            category = lookup_dict[heading_number]
            # Keep track of which number was already responded
            check_dict[heading_number] = True
            # append the response based on the category key 
            submission_dict[category].append(line.strip()[2:].strip())
        # Look for unanswered questions and add blank to keep the dictionary size uniform
        for number in num_list:
            if check_dict[number]:
                check_dict[number]=False
            else:
                category = lookup_dict[number]
                submission_dict[category].append("")
                check_dict[number]=False
    return submission_dict
        

In [18]:
def save_data(data_dict, file_name):
    print(f"Saving the file to {file_name}")
    df = pd.DataFrame(data_dict)
    df.to_csv(file_name)
    print("Success! Saved")


In [19]:
if __name__ == "__main__":
    data = main()

Authentication on going.....
Success! Authenticated
working on https://www.reddit.com/r/Accounting/comments/o6c81e/pwc_2021_compensation_thread/
populating the lists
Lists populated
Starting the loop
Saving the file to PwC_2021_Compensation_Thread.csv
Success! Saved


,Market/Office,Trust or Consulting Solutions and LOS/Vertical,"CY Level -> FY22 Level (A1>A2, S1->S2, S3->M1, etc)",Rating,Old Salary -> New Salary,Bonus,Interesting notes on what RLs/RPs have told you related to future comp.,"Anything else? (opinions on the cohort model for all LOS, opinions on the new equation, etc)"
0,HCOL (BANW),Audit,A2 -> S1,Rating not communicated yet,"66,000 -> 95,700 (45%)",Bonus not communicated yet,N/A,Pretty happy with the pay bump even factoring ...
1,NY Metro,Tax - Real Estate,A1 -> A2 (?) (Does 6 months to the end of the ...,Rating - 3,"$61,000 - $77,000 (26%)","Bonus: 2.1% based on $61,000 salary",N/A,Pretty shocked I got such a high raise with ju...
2,NYM,Advisory - Deals,"Don't know yet, but I'd be surprised if it was...",112k --> 150k (33%+),"Don't know yet, but T1 get 18.5%",N/A,Was really surprised it was this much. I'm a ...,
3,Southeast,"Consulting, Risk and Reg",S3 > S4,1 rating for second consecutive year.,93k > 104k,"$16,700",I did not get promoted “because there were no ...,
4,NYM,Assurance,A1 -> A2,Tier 3,61k -> 70k,2%,Higher ups are aware it’s been perpetual busy ...,"I’m fine with the cohort model, hard to compla..."
